In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
list_city_dt = []

for file_in in os.listdir(r'../../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
print(list_city_dt)
print(len(list_city_dt))

['paris_db=4.00', 'berlin_db=2.00', 'paris_db=1.00', 'lyon_db=1.00', 'frankfurt_db=4.00', 'frankfurt_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'paris_db=2.00', 'berlin_db=1.00', 'lyon_db=0.50', 'paris_db=0.50', 'berlin_db=0.50', 'lyon_db=4.00', 'lyon_db=2.00']
16


In [23]:
list_results = []
dict_workforce = {}
dict_workforce['instance'] = []
dict_workforce['OC'] = []
dict_workforce['model'] = []
dict_workforce['max_n_shifts'] = []
dict_workforce['workforce_size'] = []
dict_workforce['objective_value'] = []
dict_workforce['workforce_size_next'] = []
dict_workforce['objective_value_next'] = []

workforce_dict = {0:5, 1:5, 2:5, 3:5, 4:5}
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

# for base_file in list_city_dt:
for base_file in ['berlin_db=1.00']:
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"
    
    # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for OC in [1.5]:
        # for model in ['fixed', 'flex', 'partflex']:
        for model in ['fixed']:
            if model == 'partflex':
                for max_n_shifts in range(2,5):
                    objvalprev = 1
                    objval = 0
                    for trial_size in range(1, 10000):
                        workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                        if (objval < objvalprev)&(np.isnan(objval) == False):
                            objvalprev = objval
                            dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                            objval = dict_results['objective_value'][0]
                            df_ = pd.DataFrame(dict_results)
                            df_['workforce_size_trial'] = trial_size
                            list_results.append(df_)
                        else:
                            #keep track of optimal
                            dict_workforce['instance'].append(base_file)
                            dict_workforce['OC'].append(OC)
                            dict_workforce['model'].append(model)
                            dict_workforce['max_n_shifts'].append(max_n_shifts)
                            trial_size = trial_size - 1
                            dict_workforce['workforce_size'].append(trial_size)
                            dict_workforce['objective_value'].append(objvalprev)
                            dict_workforce['objective_value_next'].append(objval)
                            trial_size = 10000000000
            else:
                objvalprev = 1
                objval = 0
                for trial_size in range(1, 6):
                    print("trial_size",trial_size)
                    workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                    if (objval < objvalprev)&(np.isnan(objval) == False):
                        objvalprev = objval
                        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
                        objval = dict_results['objective_value'][0]
                        print("objective_value",objval)
                        df_ = pd.DataFrame(dict_results)
                        df_['workforce_size_trial'] = trial_size
                        list_results.append(df_)
                    else: 
                        #keep track of optimal (trial size - 1)
                        dict_workforce['instance'].append(base_file)
                        dict_workforce['OC'].append(OC)
                        dict_workforce['model'].append(model)
                        dict_workforce['max_n_shifts'].append(np.nan)
                        trial_size = trial_size - 1
                        dict_workforce['workforce_size'].append(trial_size)
                        dict_workforce['objective_value'].append(objvalprev)
                        dict_workforce['objective_value_next'].append(objval)
                        trial_size = 10000000000


trial_size 1


FileNotFoundError: [Errno 2] No such file or directory: '../instances/berlin_db=1.00_dt=doublepeak.json'

In [22]:
dict_workforce

{'instance': ['berlin_db=1.00',
  'berlin_db=1.00',
  'berlin_db=1.00',
  'berlin_db=1.00',
  'berlin_db=1.00'],
 'OC': [1.5, 1.5, 1.5, 1.5, 1.5],
 'model': ['fixed', 'fixed', 'fixed', 'fixed', 'fixed'],
 'max_n_shifts': [nan, nan, nan, nan, nan],
 'workforce_size': [0, 1, 2, 3, 4],
 'objective_value': [100000000, 100000000, 100000000, 100000000, 100000000],
 'workforce_size_next': [],
 'objective_value_next': [0, 0, 0, 0, 0]}